# Introduction

- Task: Multi-class Classification    
- Method: Deep Neural Network
- Library: PyTorch
- Dataset: CIFAR-10
    - https://www.cs.toronto.edu/~kriz/cifar.html
    - The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    - Classes: {airplain, automobile, bird, cat, deer, dog, frog, horse, ship, truck}

# Set Environment

In [1]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Selected device:", device)

torch.manual_seed(42)

Selected device: cuda


# Process Data

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10('/home/june/__github__/playground/data',
                                 download = True, train = True, transform = transform)

test_dataset = datasets.CIFAR10('/home/june/__github__/playground/data',
                                download = True, train = False, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
batch_size = 512

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                          shuffle=True, num_workers = 0)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                         shuffle = False, num_workers = 0)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Define Model

## Model Structure

In [6]:
class LinearClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view((x.size(0), -1))
        x = self.fc2(self.fc1(x))
        return x

## Hyperparameter & Functions

In [7]:
input_size = 3*32*32
hidden_size = 1024
num_classes = 10

model = LinearClassifier(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
model.to(device)
criterion.to(device)

CrossEntropyLoss()

# Train

In [9]:
def train_model(model, cirterion, optimizer, trainloader, num_epochs):
    print("-----Training Started------")
    for epoch in range(num_epochs):

        model.train()

        running_loss = 0.0
        
        for images, labels in trainloader: #tqdm
            
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(trainloader.dataset)
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        #print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        
        torch.save(model, './model/Linear.pth')
        
    print("-----Training Completed-----")

In [10]:
num_epochs = 128

train_model(model, criterion, optimizer, trainloader, num_epochs)
writer.flush()
writer.close()

-----Training Started------
-----Training Completed-----


# Test

In [11]:
def test_model(model, testloader):
    
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(testloader):
            
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy:.4f}")

In [12]:
test_model(model, testloader)

100%|██████████| 20/20 [00:01<00:00, 17.15it/s]

Accuracy on test set: 0.4131


# Inference